In [1]:
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

_____

# Process Data methods

In [2]:
def processDataLSA(pathTXT):

    acceptance_pb = []
    acc_best_sol = []
    temp = []
    best_y = []
    curr_y = []
    runtime = 0
    iteration_nbr = 0


    with open(pathTXT, "r") as file:
        dimension = file.readline().count('BEST x')
        best_xs = np.zeros((dimension, 0))
        curr_xs = np.zeros((dimension, 0))

        for line in file:
            if(line.startswith("Runtime")):
                runtime = float(re.sub("[^0-9^.]", "", line))
            elif(line.startswith("Nbr")):
                iteration_nbr = int(re.sub("[^0-9]", "", line))
            else:
                final = 5
                a = line.split("|")
                a = [i.strip() for i in a][:-1]
                acceptance_pb.append(float(a[0]))
                acc_best_sol.append(a[1])
                temp.append(float(a[2]))
                best_y.append(float(a[3]))
                curr_y.append(float(a[4]))

                best_xs_temp = np.zeros((dimension, 1))
                curr_xs_temp = np.zeros((dimension, 1))
                for i in range(dimension):
                    index = i*2
                    best_xs_temp[i] = float(a[final+index])
                    curr_xs_temp[i] = float(a[final+index+1])
                best_xs = np.concatenate((best_xs, best_xs_temp), axis = 1)
                curr_xs = np.concatenate((curr_xs, curr_xs_temp), axis = 1)
                
    acceptance_pb = np.asarray(acceptance_pb)
    acc_best_sol = np.asarray(acc_best_sol)
    temp = np.asarray(temp)
    best_y = np.asarray(best_y)
    curr_y = np.asarray(curr_y)
    
    return acceptance_pb, acc_best_sol, temp, best_y, curr_y, best_xs, curr_xs, runtime, iteration_nbr

In [3]:
def processMarkovLen(init):
    temp = []
    for i, v in enumerate(init):
        if(i>1):
            if(v==1):
                temp.append(init[i-1])
    temp.append(v)
    
    final = list(np.ones(int(temp[0]+1))*temp[0])
    for i in range(len(temp)-1):
        final += list(np.linspace(int(temp[i]), int(temp[i+1]), int(temp[i+1]+1)))[1:]
    return np.asarray(final)

In [4]:
def processDataDSA(pathTXT):
    
    acceptance_pb = []
    acc_best_sol = []
    temp = []
    density = []
    markov_len = []
    best_y = []
    curr_y = []
    runtime = 0
    iteration_nbr = 0
    
    with open(pathTXT, "r") as file:
        dimension = file.readline().count('BEST x')
        best_xs = np.zeros((dimension, 0))
        curr_xs = np.zeros((dimension, 0))

        for line in file:
            if(line.startswith("Runtime")):
                runtime = float(re.sub("[^0-9^.]", "", line))
            elif(line.startswith("Nbr")):
                iteration_nbr = int(re.sub("[^0-9]", "", line))
            else:
                final = 7
                a = line.split("|")
                a = [i.strip() for i in a][:-1]
                acceptance_pb.append(float(a[0]))
                acc_best_sol.append(a[1])
                temp.append(float(a[2]))
                density.append(float(a[3]))
                markov_len.append(float(a[4]))
                best_y.append(float(a[5]))
                curr_y.append(float(a[6]))

                best_xs_temp = np.zeros((dimension, 1))
                curr_xs_temp = np.zeros((dimension, 1))
                for i in range(dimension):
                    index = i*2
                    best_xs_temp[i] = float(a[final+index])
                    curr_xs_temp[i] = float(a[final+index+1])
                best_xs = np.concatenate((best_xs, best_xs_temp), axis = 1)
                curr_xs = np.concatenate((curr_xs, curr_xs_temp), axis = 1)
                
    acceptance_pb = np.asarray(acceptance_pb)
    acc_best_sol = np.asarray(acc_best_sol)
    temp = np.asarray(temp)
    best_y = np.asarray(best_y)
    curr_y = np.asarray(curr_y)
    
    density = np.asarray(density)
    markov_len = processMarkovLen(markov_len)[:len(temp)]
    
    
    return acceptance_pb, acc_best_sol, temp, density, markov_len, best_y, curr_y, best_xs, curr_xs, runtime, iteration_nbr

In [5]:
#path = "../java/SimulatedAnnealing/GraphData/DSA_MinFunction.txt"
#acceptance_pb, acc_best_sol, temp, density, markov_len, best_y, curr_y, best_xs, curr_xs, runtime, iteration_nbr = processDataDSA(path)
#print(len(markov_len), len(temp))

In [6]:
import pandas as pd

def processData4D_panda(pathTXT):
    a = []
    tab = []
    i = 0
    with open(pathTXT, "r") as file:
        for line in file:
            if(line.startswith("Runtime")):
                runtime = float(re.sub("[^0-9^.]", "", line))
            elif(line.startswith("Nbr")):
                iteration_nbr = int(re.sub("[^0-9]", "", line))
            else:
                a = line.split("|")
                a.pop(1)
                a = [i.strip() for i in a][:-1]
                if(i!=0):
                    a = [float(i) for i in a]
                tab.append(a)
            i+=1
    titles = tab[0]
    col_to_del = titles[:4]
    dtf = pd.DataFrame(data=tab[1:],columns=titles)
    afterdrop_all = dtf.drop(col_to_del, axis = 1)
    col_to_del = titles[4::2]
    afterdrop_cur = afterdrop_all.drop(col_to_del, axis = 1)
    col_to_del = titles[5::2]
    afterdrop_bes = afterdrop_all.drop(col_to_del, axis = 1)
    
    return afterdrop_all, afterdrop_cur, afterdrop_bes